In [ ]:
!pip install gdown

In [ ]:
# natural image
!gdown --id 1-7z0lFddFDcy97On7vO5MlucFUiFJYi1

# adv image
!gdown --id 13ukjGjYmHcxRV94vLccZ6vj_cGyO4CW4

In [ ]:
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as Data
import torch.nn.functional as F

import torchvision.utils
from torchvision import models
import torchvision.datasets as dsets
import torchvision.transforms as transforms

In [ ]:
# set device
use_cuda = True
device = torch.device("cuda" if use_cuda else "cpu")
print('device', device)

In [ ]:
# load natural image
nat_data = torch.load('lab_img.pt')

In [ ]:
# load adv image
adv_data = torch.load('pgd-resnet101-advs.pt', map_location=torch.device('cpu'))

In [ ]:
# load resnet101
model = models.resnet101(pretrained=True)

# use cuda if available
if torch.cuda.is_available():
    model.cuda()

model = model.eval()

print('use cuda', torch.cuda.is_available())

In [ ]:
# set radius
f_n_radius = 0.2  # first radius
s_n_radius = 0.3  # second radius

In [ ]:
# first test Gaussian noise --> Natural image
f_nat_stats = []
for lab, img in nat_data:
    img = img.to(device)
    diff = torch.norm(
        F.softmax(model(img)) - 
        F.softmax(model(img + f_n_radius * torch.randn_like(img))), 1).item()
    f_nat_stats.append(diff)

In [ ]:
# first test Gaussian noise --> adv image
f_adv_stats = []
for lab, img in adv_data:
    img = img.to(device)
    diff = torch.norm(
        F.softmax(model(img)) - 
        F.softmax(model(img + f_n_radius * torch.randn_like(img))), 1).item()
    f_adv_stats.append(diff)

In [ ]:
# second test Gaussian noise --> Natural image
s_nat_stats = []
for lab, img in nat_data:
    img = img.to(device)
    diff = torch.norm(
        F.softmax(model(img)) - 
        F.softmax(model(img + s_n_radius * torch.randn_like(img))), 1).item()
    s_nat_stats.append(diff)

In [ ]:
# second test Gaussian noise --> adv image
s_adv_stats = []
for lab, img in adv_data:
    img = img.to(device)
    diff = torch.norm(
        F.softmax(model(img)) - 
        F.softmax(model(img + s_n_radius * torch.randn_like(img))), 1).item()
    s_adv_stats.append(diff)

In [ ]:
# plot distribution
import matplotlib.pyplot as plt
plot1 = plt.figure(1)
plt.scatter([i for i in range(100)], f_nat_stats, color='red', label='natural image')
plt.scatter([i for i in range(100)], f_adv_stats, color='blue', label='adverarial image')
plt.title(f'{f_n_radius} radius')

plot2 = plt.figure(2)
plt.scatter([i for i in range(100)], s_nat_stats, color='red', label='natural image')
plt.scatter([i for i in range(100)], s_adv_stats, color='blue', label='adverarial image')
plt.title(f'{s_n_radius} radius')

plt.legend()

In [ ]:
# set boundary and repetition
bd = 84
n = 5

In [ ]:
# find suitable first seperation
seps = []
for i in range(n):
    nat_stats = []
    for lab, img in nat_data:
        img = img.to(device)
        diff = torch.norm(
            F.softmax(model(img)) - 
            F.softmax(model(img + f_n_radius * torch.randn_like(img))), 1).item()
        nat_stats.append(diff)
    sep = sorted(nat_stats)[bd]
    seps.append(sep)
f_mean_sep = sum(seps) / n
print(f'{f_n_radius} mean sep', f_mean_sep)    

In [ ]:
# find suitable second seperation
seps = []
for i in range(n):
    nat_stats = []
    for lab, img in nat_data:
        img = img.to(device)
        diff = torch.norm(
            F.softmax(model(img)) - 
            F.softmax(model(img + s_n_radius * torch.randn_like(img))), 1).item()
        nat_stats.append(diff)
    sep = sorted(nat_stats)[bd]
    seps.append(sep)
s_mean_sep = sum(seps) / n
print(f'{s_n_radius} mean sep', s_mean_sep)   

In [ ]:
# FPR
total = 0
wrong = 0
for lab, img in nat_data:
    img = img.to(device)
    f_diff = torch.norm(
        F.softmax(model(img)) - 
        F.softmax(model(img + f_n_radius * torch.randn_like(img))), 1).item()
    s_diff = torch.norm(
        F.softmax(model(img)) -
        F.softmax(model(img + s_n_radius * torch.randn_like(img))), 1).item()
    total += 1
    if f_diff > f_mean_sep and s_diff > s_mean_sep:
        wrong += 1
print('FPR', wrong / total)

In [ ]:
# TPR
total = 0
correct = 0
for lab, img in adv_data:
    img = img.to(device)
    f_diff = torch.norm(
        F.softmax(model(img)) - 
        F.softmax(model(img + f_n_radius * torch.randn_like(img))), 1).item()
    s_diff = torch.norm(
        F.softmax(model(img)) -
        F.softmax(model(img + s_n_radius * torch.randn_like(img))), 1).item()
    total += 1
    if f_diff > f_mean_sep and s_diff > s_mean_sep:
        correct += 1
print('TPR', correct / total)